© Alexander Jüstel, Fraunhofer IEG, Institution for Energy Infrastructures and Geothermal Systems, RWTH Aachen University,
GNU Lesser General Public License v3.0


This notebook was created by Alexander Jüstel and is part of the DGE Rollout Heat Demand Mapping project carried out by students of RWTH Aachen University within the framework of their master thesis. 


# Statistical Evaluation of the commercial Heat demand data

This notebook illustrates the statistical evaluation on a regional scale (NUTS1) for the commercial heat demand using `rasterstats` and `pandas`.


# Calculating zonal statistics for commercial heat demand

The zonal statistics will be calculated using a shape file containing one polygon for each country and the raster containing the commercial heat demand.

# Importing Libraries

In [1]:
# Importing Libraries
from rasterstats import zonal_stats
import pandas as pd
import geopandas as gpd

In [2]:
# Calculating zonal statistics
stats_com_heat = zonal_stats("../../../data/heat_demand_mapping/shape/Interreg_NWE_NUTS1_3034.shp", 
                             "../../../data/heat_demand_mapping/tif_results/NWE_HD_tert.tif",
                             stats="count min mean max median sum std",
                             geojson_out=True)


## Storing the properties in a list 

All Properties will be stored in a list. The name of the countries will be stored as well to identify the data set later on. The values for Scotland will be set to zero as no commercial heat demand was reported for this country. 

In [3]:
# Setting all values for Scotland to 0
stats_com_heat[4]['properties']['min'] = 0
stats_com_heat[4]['properties']['max'] = 0
stats_com_heat[4]['properties']['mean'] = 0
stats_com_heat[4]['properties']['count'] = 0
stats_com_heat[4]['properties']['sum'] = 0
stats_com_heat[4]['properties']['std'] = 0
stats_com_heat[4]['properties']['median'] = 0
stats_com_heat[4]['properties']

OrderedDict([('CNTR_CODE', 'UK'),
             ('NAME_LATN', 'Wales'),
             ('area_m2', 20805868751.46),
             ('Region', 'UKL'),
             ('min', 0),
             ('max', 0),
             ('mean', 0),
             ('count', 0),
             ('sum', 0),
             ('std', 0),
             ('median', 0)])

In [4]:
# Creating list of properties dicts
properties_com_heat = [stats_com_heat[i]['properties'] for i in range(len(stats_com_heat))]
properties_com_heat[:2]

[OrderedDict([('CNTR_CODE', 'FR'),
              ('NAME_LATN', 'Pays de la Loire'),
              ('area_m2', 32328662083.881),
              ('Region', 'FR-PDL'),
              ('min', 0.001),
              ('max', 38894.75),
              ('mean', 64.1960597956392),
              ('count', 124290),
              ('sum', 7978928.271999997),
              ('std', 313.2736042295819),
              ('median', 10.529)]),
 OrderedDict([('CNTR_CODE', 'FR'),
              ('NAME_LATN', 'Bretagne'),
              ('area_m2', 27352052071.085),
              ('Region', 'FR-BRE'),
              ('min', 0.001),
              ('max', 13364.74),
              ('mean', 51.02850850995504),
              ('count', 112104),
              ('sum', 5720499.918),
              ('std', 203.61467249233715),
              ('median', 8.612)])]

## Creating a Dataframe from dicts

A dataframe containing all statistical values is created. 

In [5]:
# Creating DataFrame from list of dicts
df_com_heat = pd.DataFrame(properties_com_heat, index=range(36))

# Creating a column called 'Region'
#df_com_heat['Region'] = df_com_heat['NAME_LATN']


## Cumulated Heat demand of the entire Interreg NWE region

The cumulated heat demand is equal to the `sum` of all country values in `MWh`converted to `TWh`. In addition, the column `sum` is equal to the `Combined/Total HD [TWh]`. The `Share of Total HD [%]` is the `Combined/Total HD [TWh]` divided by the cumulated heat demand and converted to `TWh`.

In [6]:
# Creating a new column called 'Commerical HD [TWh]'
df_com_heat['Commercial HD [TWh]'] = df_com_heat['sum']/1e6

# Calculating the commercial cumulated heat demand of the Interreg NWE Region
df_com_heat_cumulated_hd = round(sum(df_com_heat['Commercial HD [TWh]']),1)

# Calculating the share of national heat demand to the commercial heat demand of the NWE Region
df_com_heat['Share of Commercial HD [%]'] = df_com_heat['Commercial HD [TWh]']/df_com_heat_cumulated_hd*100

print(str(df_com_heat_cumulated_hd) + ' TWh')

552.0 TWh


## Calculating the area that is covered by heat demand squares

The are that is covered by heat is equal to the `count` of pixels of the raster times the resolution (100 m * 100 m) times the conversion factor from m2 to km2. In addition, the `Average of heated area total [MWh/ha]` is equal to `mean`. 

In [7]:
# Calculating the area that is covered by heat demand squares
df_com_heat['Area of heat demand [km2]'] = df_com_heat['count']*100*100*1e-6

# Calculating the total Interreg NWE region where a heat demand was calculated
total_area_of_heat_demand = round(sum(df_com_heat['Area of heat demand [km2]']))

# Creating a new column called 'Average of heated area commercial [MWh/ha]'
df_com_heat['Average of heated area of Commercial HD [MWh/ha]'] = df_com_heat['mean']


print(str(total_area_of_heat_demand) + ' km2')
df_com_heat

77514 km2


,CNTR_CODE,NAME_LATN,area_m2,Region,min,max,mean,count,sum,std,median,Commercial HD [TWh],Share of Commercial HD [%],Area of heat demand [km2],Average of heated area of Commercial HD [MWh/ha]
0,FR,Pays de la Loire,3.232866e+10,FR-PDL,1.000000e-03,38894.750000,64.196060,124290,7.978928e+06,313.273604,10.529000,7.978928,1.445458,1242.90,64.196060
1,FR,Bretagne,2.735205e+10,FR-BRE,1.000000e-03,13364.740000,51.028509,112104,5.720500e+06,203.614672,8.612000,5.720500,1.036322,1121.04,51.028509
2,UK,South West (England),2.389106e+10,UKK,1.057726e-08,1282.395988,32.866141,314988,1.035244e+07,58.142261,8.283615,10.352440,1.875442,3149.88,32.866141
3,UK,South East (England),1.910486e+10,UKJ,4.187370e-10,1624.323865,38.547878,439564,1.694426e+07,69.229013,12.368036,16.944259,3.069612,4395.64,38.547878
4,UK,Wales,2.080587e+10,UKL,0.000000e+00,0.000000,0.000000,0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,IE,Ireland,6.973779e+10,IE0,1.225298e-08,2268.137290,22.719253,456912,1.038070e+07,60.804701,2.735942,10.380699,1.880561,4569.12,22.719253
6,NL,West-Nederland,1.002304e+10,NL3,6.909057e-08,5894.980860,147.346715,78280,1.153430e+07,337.852293,37.342189,11.534301,2.089547,782.80,147.346715
7,UK,Northern Ireland,1.412807e+10,UKN,6.549756e-09,1247.323776,17.570394,171473,3.012848e+06,46.212376,2.451345,3.012848,0.545806,1714.73,17.570394
8,UK,Scotland,7.868410e+10,UKM,7.388853e-03,54.658416,11.885017,11,1.307352e+02,16.799623,3.939400,0.000131,0.000024,0.11,11.885017
9,FR,Hauts-de-France,3.192705e+10,FR-HDF,2.910587e-06,29798.757000,78.895231,160214,1.264012e+07,305.497747,12.211000,12.640121,2.289877,1602.14,78.895231


In [8]:
# Calculating the average heat demand of the heated area 
average_of_heated_area_total = sum(df_com_heat['mean'])/9

print(average_of_heated_area_total)

569.5628587695071


In [9]:
# Loading the Interreg NWE Region as Shape File
gdf_interreg_nwe = gpd.read_file("../../../data/heat_demand_mapping/shape/Interreg_NWE_NUTS1_3034.shp")

# Adding the area im m2 to the gdf
gdf_interreg_nwe['area'] = gdf_interreg_nwe.area

# Converting the area in m2 to km2
gdf_interreg_nwe['Area (planimetric) [km2]'] = round(gdf_interreg_nwe['area']*1e-6)

# Changing the data type of the column to int
gdf_interreg_nwe['Area (planimetric) [km2]'] = gdf_interreg_nwe['Area (planimetric) [km2]'].astype(int)

# Calculating the total planimetric area of the Interreg NWE Region
gdf_interreg_nwe_total_area_planimetric = sum(gdf_interreg_nwe['Area (planimetric) [km2]'])

# Adding the ellipsoidal areas manually to the DataFrame, values were calculated using QGIS
gdf_interreg_nwe['Area (ellipsoidal) [km2]']= round(gdf_interreg_nwe['area_m2']*1e-6)

# Calculating the total ellipsoidal area of the Interreg NWE Region
gdf_interreg_nwe_total_area_ellipsoid = sum(gdf_interreg_nwe['Area (ellipsoidal) [km2]'])

# Calculating the share of each country to the total ellipsoidal Interreg NWE Region
gdf_interreg_nwe['Share of Total Area [%]'] = gdf_interreg_nwe['Area (ellipsoidal) [km2]']/gdf_interreg_nwe_total_area_ellipsoid*100

print(str(gdf_interreg_nwe_total_area_planimetric) + ' km2')
print(str(gdf_interreg_nwe_total_area_ellipsoid) + ' km2')
gdf_interreg_nwe

788012 km2
842823.0 km2


,CNTR_CODE,NAME_LATN,area_m2,Region,geometry,area,Area (planimetric) [km2],Area (ellipsoidal) [km2],Share of Total Area [%]
0,FR,Pays de la Loire,3.232866e+10,FR-PDL,"MULTIPOLYGON (((3088284.755 2310490.484, 30883...",3.024839e+10,30248,32329.0,3.835799
1,FR,Bretagne,2.735205e+10,FR-BRE,"MULTIPOLYGON (((2930641.614 2531547.043, 29306...",2.555848e+10,25558,27352.0,3.245284
2,UK,South West (England),2.389106e+10,UKK,"MULTIPOLYGON (((2874711.281 2699278.507, 28747...",2.227825e+10,22278,23891.0,2.834640
3,UK,South East (England),1.910486e+10,UKJ,"MULTIPOLYGON (((3235335.763 2725588.105, 32354...",1.781559e+10,17816,19105.0,2.266787
4,UK,Wales,2.080587e+10,UKL,"MULTIPOLYGON (((3022516.409 2859385.211, 30225...",1.941505e+10,19415,20806.0,2.468608
5,IE,Ireland,6.973779e+10,IE0,"MULTIPOLYGON (((2651794.967 2994393.242, 26518...",6.514953e+10,65150,69738.0,8.274335
6,NL,West-Nederland,1.002304e+10,NL3,"MULTIPOLYGON (((3667957.187 2938854.312, 36680...",9.350850e+09,9351,10023.0,1.189218
7,UK,Northern Ireland,1.412807e+10,UKN,"MULTIPOLYGON (((2942109.196 3254113.370, 29421...",1.323239e+10,13232,14128.0,1.676271
8,UK,Scotland,7.868410e+10,UKM,"MULTIPOLYGON (((3096602.179 3293529.058, 30965...",7.423733e+10,74237,78684.0,9.335768
9,FR,Hauts-de-France,3.192705e+10,FR-HDF,"MULTIPOLYGON (((3406023.264 2627594.642, 34060...",2.977877e+10,29779,31927.0,3.788103


## Calculating Average total heat demand

In [10]:
# Calculating the average commercial heat demand including not heated areas
average_total_heat_demand = df_com_heat_cumulated_hd/gdf_interreg_nwe_total_area_ellipsoid * 1e4

print(str(average_total_heat_demand)+' MWh/ha')

6.54941784929932 MWh/ha


## Merge DataFrames

The two DataFrames are now being merged. In addition, a row for the values of the entire Interreg NWE Region is added.

In [11]:
# Merging DataFrames
gdf_com_heat = pd.merge(df_com_heat, gdf_interreg_nwe, 
                        on=['NAME_LATN', 'Region'], 
                        how='left')

# Calculating the average HD demand per hectare including areas that are not heated 
gdf_com_heat['Average of Commercial HD [MWh/ha]'] = gdf_com_heat['Commercial HD [TWh]']*1e6/gdf_com_heat['Area (ellipsoidal) [km2]']/100

# Calculating the share of heated heat demand area to the total area
gdf_com_heat['Share of heated heat demand area [%]'] = gdf_com_heat['Area of heat demand [km2]']/gdf_com_heat['Area (ellipsoidal) [km2]']*100

# Creating a dict summarizing the the calculated values for the entire Interreg NWE Region
total_interreg = {'Region': 'Interreg NWE', 
                  'Area (planimetric) [km2]': gdf_interreg_nwe_total_area_planimetric,
                  'Area (ellipsoidal) [km2]': gdf_interreg_nwe_total_area_ellipsoid,
                  'Share of Total Area [%]': 100, 
                  'Commercial HD [TWh]': df_com_heat_cumulated_hd,
                  'Share of Commercial HD [%]': 100, 
                  'Average of Commercial HD [MWh/ha]': average_total_heat_demand,
                  'Average of heated area of Commercial HD [MWh/ha]': average_of_heated_area_total, 
                  'Area of heat demand [km2]': total_area_of_heat_demand,
                  'Share of heated heat demand area [%]': total_area_of_heat_demand/gdf_interreg_nwe_total_area_ellipsoid*100}

# Appending the row to the DataFrame
gdf_com_heat = gdf_com_heat.append(total_interreg,ignore_index=True)

# Selecting columns
gdf_com_heat = gdf_com_heat[['Region', 'Area (planimetric) [km2]', 
                             'Area (ellipsoidal) [km2]', 
                             'Share of Total Area [%]', 
                             'Commercial HD [TWh]', 
                             'Share of Commercial HD [%]', 
                             'Average of Commercial HD [MWh/ha]', 
                             'Average of heated area of Commercial HD [MWh/ha]', 
                             'Area of heat demand [km2]',
                             'Share of heated heat demand area [%]']].sort_values(by='Commercial HD [TWh]', ascending=False).reset_index().drop('index', axis=1)
gdf_com_heat.round(decimals=1)#.style.applymap(lambda x: "background-color: red" if x>0 else "background-color: green")

,Region,Area (planimetric) [km2],Area (ellipsoidal) [km2],Share of Total Area [%],Commercial HD [TWh],Share of Commercial HD [%],Average of Commercial HD [MWh/ha],Average of heated area of Commercial HD [MWh/ha],Area of heat demand [km2],Share of heated heat demand area [%]
0,Interreg NWE,788012,842823.0,100.0,552.0,100.0,6.5,569.6,77514.0,9.2
1,DEA,31807,34106.0,4.0,82.5,14.9,24.2,107.4,7683.2,22.5
2,DE1,33587,35961.0,4.3,73.6,13.3,20.5,99.5,7399.9,20.6
3,DE7,19678,21102.0,2.5,42.5,7.7,20.1,109.0,3896.0,18.5
4,FR1,11268,12068.0,1.4,37.2,6.7,30.8,247.4,1503.0,12.5
5,DEB,18518,19853.0,2.4,31.3,5.7,15.8,83.7,3735.5,18.8
6,CH0,38663,41263.0,4.9,26.9,4.9,6.5,96.3,2788.5,6.8
7,DEC,2400,2573.0,0.3,23.1,4.2,89.9,1155.8,200.0,7.8
8,BE2,12678,13597.0,1.6,19.5,3.5,14.3,1324.7,146.9,1.1
9,UKJ,17816,19105.0,2.3,16.9,3.1,8.9,38.5,4395.6,23.0


In [12]:
gdf_com_heat.round(decimals=1).to_csv('Statistics_Commercial_Heat_Demand_NUTS1.csv', index=False)